In [1]:
#imports and setup
import pandas as pd
import os

import sys
sys.path.append('./dataSets/')

from state_abbrevs import states

pd.set_option('display.max_columns', 500)

ModuleNotFoundError: No module named 'state_abbrevs'

In [2]:
base_dir = './dataSets/income/'
filenames = sorted(os.listdir(base_dir))
print(filenames)
data_frames = []
for filename in filenames:
    df = pd.read_csv(base_dir + filename, encoding='latin-1', )
    data_frames.append(df)

['1969_79_89.csv', '1999.csv', '2009.csv', '2010.csv', '2011.csv', '2012.csv', '2013.csv', '2014.csv', '2015.csv']


In [3]:
def full_state_name(county):
    ct = county.split(',')
    full_name = states[ct[1].lstrip()]
    return ct[0] + ', ' + full_name

left = data_frames[0][['county','1969', '1979', '1989']]
left['county'] = left['county'].apply(full_state_name)
left.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,county,1969,1979,1989
0,"Autauga County, Alabama","6,808","16,524","28,337"
1,"Baldwin County, Alabama","6,430","14,614","25,712"
2,"Barbour County, Alabama","4,318","10,058","19,389"
3,"Bibb County, Alabama","4,935","12,299","19,775"
4,"Blount County, Alabama","5,572","12,657","22,382"


In [4]:
data_frames[1] = data_frames[1].rename(index=str, columns=({ 'GEO.id2' : 'FIPS', 'GEO.display-label' : 'county', 'VD02' : '1999'}))
right = data_frames[1][['GEO.id', 'FIPS', 'county', '1999']]
result = pd.merge(left, right, on='county')
result.head()

,county,1969,1979,1989,GEO.id,FIPS,1999
0,"Autauga County, Alabama","6,808","16,524","28,337",0500000US01001,1001,41953
1,"Baldwin County, Alabama","6,430","14,614","25,712",0500000US01003,1003,40174
2,"Barbour County, Alabama","4,318","10,058","19,389",0500000US01005,1005,25080
3,"Bibb County, Alabama","4,935","12,299","19,775",0500000US01007,1007,31402
4,"Blount County, Alabama","5,572","12,657","22,382",0500000US01009,1009,35054


In [5]:
# data_frames[2].head()
offset=2
for idx, df in enumerate(data_frames[offset:]):
    pos = offset + idx
    year = filenames[pos][:-4]
    if year == '2009':
        key = 'HC01_EST_VC12'
    else:
        key = 'HC01_EST_VC13'
    df_year = df.rename(index=str, columns=({
        'GEO.id2' : 'FIPS',
         key : year
    }))
#     print(right.head())
    df_year = df_year[['FIPS', year]]
    result = pd.merge(result, df_year, on='FIPS')

result.head()

,county,1969,1979,1989,GEO.id,FIPS,1999,2009,2010,2011,2012,2013,2014,2015
0,"Autauga County, Alabama","6,808","16,524","28,337",0500000US01001,1001,41953,51463,53255,53899,53773,53682,52475,51281.0
1,"Baldwin County, Alabama","6,430","14,614","25,712",0500000US01003,1003,40174,48918,50147,51321,50706,50221,50183,50254.0
2,"Barbour County, Alabama","4,318","10,058","19,389",0500000US01005,1005,25080,32537,33219,34041,31889,32911,35634,32964.0
3,"Bibb County, Alabama","4,935","12,299","19,775",0500000US01007,1007,31402,41236,41770,40506,36824,36447,37984,38678.0
4,"Blount County, Alabama","5,572","12,657","22,382",0500000US01009,1009,35054,45406,45549,45404,45192,44145,44409,45813.0


In [6]:
result = result[['county', 'GEO.id', 'FIPS', '1969', '1979', '1989', '1999', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
result.head()

,county,GEO.id,FIPS,1969,1979,1989,1999,2009,2010,2011,2012,2013,2014,2015
0,"Autauga County, Alabama",0500000US01001,1001,"6,808","16,524","28,337",41953,51463,53255,53899,53773,53682,52475,51281.0
1,"Baldwin County, Alabama",0500000US01003,1003,"6,430","14,614","25,712",40174,48918,50147,51321,50706,50221,50183,50254.0
2,"Barbour County, Alabama",0500000US01005,1005,"4,318","10,058","19,389",25080,32537,33219,34041,31889,32911,35634,32964.0
3,"Bibb County, Alabama",0500000US01007,1007,"4,935","12,299","19,775",31402,41236,41770,40506,36824,36447,37984,38678.0
4,"Blount County, Alabama",0500000US01009,1009,"5,572","12,657","22,382",35054,45406,45549,45404,45192,44145,44409,45813.0


In [7]:
result.to_csv('./dataSets/income_county_current.csv')